In [1]:
%matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import missingno as msno
import pandas_profiling as prof
from dfply import *
import pulp

In [2]:
fpl_df = pd.read_csv("../data/optimizer_stats.csv")

In [3]:
fpl_df.shape

(649, 50)

In [4]:
valid_formations = [[1,3,4,3], [1,3,5,2], [1,4,4,2], [1,5,3,2]]

In [5]:
fpl_df.total_points.describe()

count    649.000000
mean      32.024653
std       34.944922
min       -3.000000
25%        0.000000
50%       21.000000
75%       53.000000
max      190.000000
Name: total_points, dtype: float64

In [6]:
fpl_df.xg_net.describe()

count    488.000000
mean       0.015164
std        1.040719
min       -5.300000
25%       -0.400000
50%        0.000000
75%        0.300000
max        5.700000
Name: xg_net, dtype: float64

In [7]:
fpl_df.xa_net.describe()

count    488.000000
mean       0.057377
std        0.832839
min       -3.500000
25%       -0.300000
50%        0.000000
75%        0.200000
max        6.000000
Name: xa_net, dtype: float64

In [8]:
fpl_df = fpl_df.query("total_points > 20 or xg_net >= 0.3 or xa_net >= 0.2").reset_index()

In [9]:
fpl_df.team.unique()

array(['arsenal', 'aston_villa', 'brighton', 'burnley', 'chelsea',
       'crystal_palace', 'everton', 'fulham', 'leicester', 'leeds',
       'liverpool', 'man_city', 'man_utd', 'newcastle', 'sheffield_utd',
       'southampton', 'spurs', 'west_brom', 'west_ham', 'wolves'],
      dtype=object)

In [10]:
fpl_df[["display_slug", "team", "points_per_game"]]

,display_slug,team,points_per_game
0,David Luiz,arsenal,1.8
1,Aubameyang,arsenal,4.5
2,Cédric,arsenal,3.4
3,Lacazette,arsenal,3.9
4,Leno,arsenal,3.8
...,...,...,...
341,Fabio Silva,wolves,1.8
342,Vitinha,wolves,1.3
343,Semedo,wolves,3.0
344,Ait Nouri,wolves,2.8


In [11]:
fpl_df.columns.tolist()

['index',
 'display_slug',
 'team',
 'position_fpl',
 'points_per_game',
 'form',
 'total_points',
 'value_form',
 'selected_by_percent',
 'influence_rank_by_position',
 'creativity_rank_by_position',
 'ict_index_by_position',
 'transfers_in',
 'transfers_out',
 'goals_scored',
 'assists_fpl',
 'goals_conceded',
 'clean_sheets',
 'penalties_saved',
 'penalties_missed',
 'saves',
 'bonus',
 'bps',
 'shots_total_per90',
 'goals_per_shot',
 'xg_net',
 'xa_net',
 'passes_into_final_third',
 'passes_into_penalty_area',
 'crosses_into_penalty_area',
 'sca_per90',
 'gca_per90',
 'tackles_def_3rd',
 'tackles_att_3rd',
 'clearances',
 'errors',
 'touches_att_3rd',
 'touches_att_pen_area',
 'carry_progressive_distance',
 'minutes_90s_y',
 'minutes_per_start',
 'points_per_match',
 'cards_yellow_red',
 'fouls',
 'fouled',
 'own_goals',
 'pens_won',
 'pens_conceded',
 'own_goals.1',
 'aerials_won_pct',
 'cost']

In [12]:
fpl_df.position_fpl.sort_values().unique()

array(['DEF', 'FWD', 'GKP', 'MID'], dtype=object)

`x` is the variable that we need, since we are going to select x's.

In [13]:
def_df = fpl_df.copy()
def_players = pulp.LpVariable.dict("player", range(0, def_df.shape[0]), \
                                   lowBound=0, upBound=1, cat="Continuous")

In [14]:
len(def_players)

346

### Maximizing `total_points`

In [15]:
model = pulp.LpProblem("fpl_roaster", pulp.LpMaximize)

In [16]:
model += pulp.lpSum(def_df["total_points"] * def_players[idx] for idx in range(0, def_df.shape[0]))

### Adding constraints

In [17]:
model += (sum(def_players[idx] for idx in range(0, def_df.shape[0])) == 10, "max_number_of_players_allowed")

In [18]:
position_constraints = list(zip(def_df.position_fpl.sort_values().unique().tolist(), [3, 2, 1, 5]))
print(position_constraints)
for pos, count in position_constraints:
    model += (sum(def_players[idx] for idx in range(0, def_df.shape[0]) \
                  if def_df.loc[idx, "position_fpl"] == pos) == count, f"max_{pos.lower()}_allowed")

[('DEF', 3), ('FWD', 2), ('GKP', 1), ('MID', 5)]


In [19]:
model += (sum(def_players[idx] * def_df.loc[idx, "cost"] for idx in range(0, def_df.shape[0])\
             if def_df.loc[idx, "cost"]) == 85, "budget")

In [20]:
print(model.solve(pulp.GLPK_CMD(msg=True))) # Outputs 1 if successful
print(pulp.LpStatus[model.status]) # Outputs "Optimal" if... well, optimal
print(pulp.value(model.objective)) # Shows us our points total

-3
Undefined
0.0


In [21]:
pulp.value(model.objective)

0.0

In [22]:
for i in range(0, len(def_df)):
    if pulp.value(def_players[i]) == 1:
        print(f"{def_df['display_slug'][i]}({def_df['team'][i]}:{def_df['position_fpl'][i]}): {def_df['total_points'][i]} points.")

### Data Manipulation

We can fish out data we require for team selection from the original dataset

In [112]:
fixtures_1920_df = pd.read_csv("../data/vaastav/2019-20/fixtures.csv")

In [111]:
teams_1920_df = pd.read_csv("../data/vaastav/2019-20/teams.csv")

In [25]:
teams_1920_df.id.unique().tolist()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [26]:
teams_1920_df.columns.tolist()

['code',
 'draw',
 'form',
 'id',
 'loss',
 'name',
 'played',
 'points',
 'position',
 'pulse_id',
 'short_name',
 'strength',
 'strength_attack_away',
 'strength_attack_home',
 'strength_defence_away',
 'strength_defence_home',
 'strength_overall_away',
 'strength_overall_home',
 'team_division',
 'unavailable',
 'win']

In [27]:
print(teams_1920_df.loc[1, ["code", "id", "name", "pulse_id", "strength"]])
print(teams_2021_df.loc[2, ["code", "id", "name", "pulse_id", "strength"]])

code                  7
id                    2
name        Aston Villa
pulse_id              2
strength              2
Name: 1, dtype: object
code              36
id                 3
name        Brighton
pulse_id         131
strength           3
Name: 2, dtype: object


In [28]:
teams_2021_df.loc[0, "name"]

'Arsenal'

In [29]:
teams_1920_df.head()

,code,draw,form,id,loss,name,played,points,position,pulse_id,...,strength,strength_attack_away,strength_attack_home,strength_defence_away,strength_defence_home,strength_overall_away,strength_overall_home,team_division,unavailable,win
0,3,0,NaN,1,0,Arsenal,0,0,0,1,...,4,1170,1170,1200,1150,1240,1180,NaN,False,0
1,7,0,NaN,2,0,Aston Villa,0,0,0,2,...,2,980,970,1040,1000,1050,1020,NaN,False,0
2,91,0,NaN,3,0,Bournemouth,0,0,0,127,...,2,1030,990,1050,1000,1020,1020,NaN,False,0
3,36,0,NaN,4,0,Brighton,0,0,0,131,...,2,1100,1100,1030,1040,1010,1050,NaN,False,0
4,90,0,NaN,5,0,Burnley,0,0,0,43,...,3,1070,1130,1110,970,1180,1110,NaN,False,0


In [30]:
team_codes = teams_1920_df[["name", "id", "code"]].to_dict(orient="records")

In [110]:
sorted(team_codes, key=lambda x: x['code'])

[{'name': 'Man Utd', 'id': 12, 'code': 1},
 {'name': 'Arsenal', 'id': 1, 'code': 3},
 {'name': 'Newcastle', 'id': 13, 'code': 4},
 {'name': 'Spurs', 'id': 17, 'code': 6},
 {'name': 'Aston Villa', 'id': 2, 'code': 7},
 {'name': 'Chelsea', 'id': 6, 'code': 8},
 {'name': 'Everton', 'id': 8, 'code': 11},
 {'name': 'Leicester', 'id': 9, 'code': 13},
 {'name': 'Liverpool', 'id': 10, 'code': 14},
 {'name': 'Southampton', 'id': 16, 'code': 20},
 {'name': 'West Ham', 'id': 19, 'code': 21},
 {'name': 'Crystal Palace', 'id': 7, 'code': 31},
 {'name': 'Brighton', 'id': 4, 'code': 36},
 {'name': 'Wolves', 'id': 20, 'code': 39},
 {'name': 'Man City', 'id': 11, 'code': 43},
 {'name': 'Norwich', 'id': 14, 'code': 45},
 {'name': 'Sheffield Utd', 'id': 15, 'code': 49},
 {'name': 'Watford', 'id': 18, 'code': 57},
 {'name': 'Burnley', 'id': 5, 'code': 90},
 {'name': 'Bournemouth', 'id': 3, 'code': 91}]

In [32]:
fixtures_1920_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,stats,team_a,team_a_difficulty,team_a_score,team_h,team_h_difficulty,team_h_score
0,1059702,1,True,True,1,2019-08-09T19:00:00Z,90,False,True,"[{'h': [{'value': 1, 'element': 183}, {'value'...",14,5,1,10,2,4
1,1059709,1,True,True,8,2019-08-10T11:30:00Z,90,False,True,"[{'h': [], 'a': [{'value': 3, 'element': 214},...",11,2,5,19,4,0
2,1059703,1,True,True,2,2019-08-10T14:00:00Z,90,False,True,"[{'h': [{'value': 1, 'element': 61}], 'a': [{'...",15,2,1,3,3,1
3,1059704,1,True,True,3,2019-08-10T14:00:00Z,90,False,True,"[{'h': [{'value': 2, 'element': 90}, {'value':...",16,3,0,5,3,3
4,1059705,1,True,True,4,2019-08-10T14:00:00Z,90,False,True,"[{'h': [], 'a': [], 'identifier': 'goals_score...",8,3,0,7,2,0


In [33]:
fixtures_1920_df.tail()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,stats,team_a,team_a_difficulty,team_a_score,team_h,team_h_difficulty,team_h_score
375,1060077,47,True,True,376,2020-07-26T15:00:00Z,90,False,True,"[{'h': [], 'a': [{'value': 1, 'element': 241},...",12,3,2,9,4,0
376,1060078,47,True,True,377,2020-07-26T15:00:00Z,90,False,True,"[{'h': [{'value': 2, 'element': 215}, {'value'...",14,4,0,11,2,5
377,1060079,47,True,True,378,2020-07-26T15:00:00Z,90,False,True,"[{'h': [{'value': 1, 'element': 554}], 'a': [{...",10,2,3,13,4,1
378,1060080,47,True,True,379,2020-07-26T15:00:00Z,90,False,True,"[{'h': [{'value': 2, 'element': 437}, {'value'...",15,2,1,16,3,3
379,1060081,47,True,True,380,2020-07-26T15:00:00Z,90,False,True,"[{'h': [{'value': 1, 'element': 392}], 'a': [{...",2,2,1,19,2,1


In [34]:
[team["name"] for team in team_codes if team["id"] == 1][0]

'Arsenal'

In [35]:
def fetch_team_name(row):
    return [team["name"] for team in team_codes if team["id"] == row][0]

In [36]:
fixtures_1920_df["team_a_slug"] = fixtures_1920_df.team_a.apply(lambda x: fetch_team_name(x))
fixtures_1920_df["team_h_slug"] = fixtures_1920_df.team_h.apply(lambda x: fetch_team_name(x))

In [37]:
fixtures_1920_df.event.replace({39: 30, 40: 31, 41: 32, 42: 33, 43: 34, 44: 35, 45: 36, 46: 37, 47: 38}, inplace=True)

In [38]:
fixtures_1920_df.event.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38])

In [39]:
fixtures_1920_df.columns

Index(['code', 'event', 'finished', 'finished_provisional', 'id',
       'kickoff_time', 'minutes', 'provisional_start_time', 'started', 'stats',
       'team_a', 'team_a_difficulty', 'team_a_score', 'team_h',
       'team_h_difficulty', 'team_h_score', 'team_a_slug', 'team_h_slug'],
      dtype='object')

In [40]:
away_team_1920_df = fixtures_1920_df[["team_a_slug", "team_a_difficulty", "event"]]
home_team_1920_df = fixtures_1920_df[["team_h_slug", "team_h_difficulty", "event"]]

In [41]:
home_team_1920_df

,team_h_slug,team_h_difficulty,event
0,Liverpool,2,1
1,West Ham,4,1
2,Bournemouth,3,1
3,Burnley,3,1
4,Crystal Palace,2,1
...,...,...,...
375,Leicester,4,38
376,Man City,2,38
377,Newcastle,4,38
378,Southampton,3,38


In [42]:
away_team_1920_df

,team_a_slug,team_a_difficulty,event
0,Norwich,5,1
1,Man City,2,1
2,Sheffield Utd,2,1
3,Southampton,3,1
4,Everton,3,1
...,...,...,...
375,Man Utd,3,38
376,Norwich,4,38
377,Liverpool,2,38
378,Sheffield Utd,2,38


In [43]:
("Arsenal " * 20).split(" ")[:-1]

['Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal',
 'Arsenal']

In [44]:
away_team_1920_df_gb = away_team_1920_df.groupby(by="team_a_slug")
home_team_1920_df_gb = home_team_1920_df.groupby(by="team_h_slug")

In [45]:
def figure_groups(team_group, team, fixture):
    group = team_group[[f"team_{fixture}_difficulty"]].transpose()
    return list(zip((f"{team} " * 20).split(" ")[:-1], (f"{fixture} " * 20).split(" ")[:-1], \
                    [f"gw_{event}" for event in team_group["event"].tolist()], \
                           [fdr for fdr in group.iloc[0]]))


In [46]:
fdr_by_gw_home = []

In [47]:
for team in away_team_1920_df_gb.groups.keys():
    print(f"For team: {team}")
    fdr_by_gw_home.append(figure_groups(away_team_1920_df_gb.get_group(team), team, "a"))

For team: Arsenal
For team: Aston Villa
For team: Bournemouth
For team: Brighton
For team: Burnley
For team: Chelsea
For team: Crystal Palace
For team: Everton
For team: Leicester
For team: Liverpool
For team: Man City
For team: Man Utd
For team: Newcastle
For team: Norwich
For team: Sheffield Utd
For team: Southampton
For team: Spurs
For team: Watford
For team: West Ham
For team: Wolves


In [48]:
fdr_by_gw_away = []

In [49]:
for team in home_team_1920_df_gb.groups.keys():
    print(f"For team: {team}")
    fdr_by_gw_away.append(figure_groups(home_team_1920_df_gb.get_group(team), team, "h"))

For team: Arsenal
For team: Aston Villa
For team: Bournemouth
For team: Brighton
For team: Burnley
For team: Chelsea
For team: Crystal Palace
For team: Everton
For team: Leicester
For team: Liverpool
For team: Man City
For team: Man Utd
For team: Newcastle
For team: Norwich
For team: Sheffield Utd
For team: Southampton
For team: Spurs
For team: Watford
For team: West Ham
For team: Wolves


In [50]:
fdr_by_gw_away[0]

[('Arsenal', 'h', 'gw_2', 3),
 ('Arsenal', 'h', 'gw_4', 3),
 ('Arsenal', 'h', 'gw_6', 2),
 ('Arsenal', 'h', 'gw_8', 2),
 ('Arsenal', 'h', 'gw_10', 3),
 ('Arsenal', 'h', 'gw_11', 4),
 ('Arsenal', 'h', 'gw_13', 3),
 ('Arsenal', 'h', 'gw_15', 2),
 ('Arsenal', 'h', 'gw_17', 4),
 ('Arsenal', 'h', 'gw_20', 4),
 ('Arsenal', 'h', 'gw_21', 4),
 ('Arsenal', 'h', 'gw_23', 3),
 ('Arsenal', 'h', 'gw_26', 2),
 ('Arsenal', 'h', 'gw_27', 2),
 ('Arsenal', 'h', 'gw_29', 2),
 ('Arsenal', 'h', 'gw_32', 2),
 ('Arsenal', 'h', 'gw_34', 3),
 ('Arsenal', 'h', 'gw_36', 4),
 ('Arsenal', 'h', 'gw_38', 2)]

In [51]:
gw_data = [sorted(fdr_by_gw_home[idx] + fdr_by_gw_away[idx], key=lambda x: x[2]) \
 for idx, team in enumerate(list(home_team_1920_df_gb.groups.keys()))]

In [52]:
gw_data_df_1920 = pd.DataFrame(sum(gw_data,[]), columns=["team", "fixture", "gw", "fdr"])

In [53]:
gw_data_df_1920.head()

,team,fixture,gw,fdr
0,Arsenal,a,gw_1,2
1,Arsenal,h,gw_10,3
2,Arsenal,h,gw_11,4
3,Arsenal,a,gw_12,3
4,Arsenal,h,gw_13,3


In [54]:
players_1920_id_df = pd.read_csv("../data/vaastav/2019-20/gws/merged_gw.csv")

In [55]:
player_team_df = pd.read_csv("../data/vaastav/2019-20/players_raw.csv")[["team", "team_code", "id", "first_name", "second_name"]]

In [56]:
player_team_df["element"] = player_team_df["id"]

In [57]:
player_team_df

,team,team_code,id,first_name,second_name,element
0,1,3,1,Shkodran,Mustafi,1
1,1,3,2,Héctor,Bellerín,2
2,1,3,3,Sead,Kolasinac,3
3,1,3,4,Ainsley,Maitland-Niles,4
4,1,3,5,Sokratis,Papastathopoulos,5
...,...,...,...,...,...,...
661,20,39,587,Oskar,Buur,587
662,20,39,607,Ryan,Giles,607
663,20,39,609,Bruno André,Cavaco Jordao,609
664,20,39,619,Daniel,Castelo Podence,619


In [58]:
player_team_df.query("element == 376")

,team,team_code,id,first_name,second_name,element
596,19,21,376,Aaron,Cresswell,376


In [85]:
players_1920_with_team_det = pd.merge(players_1920_id_df, player_team_df, how="inner", on="element")

In [62]:
teams_df = pd.read_csv("../data/vaastav/2019-20/teams.csv")

In [63]:
teams_df

,code,draw,form,id,loss,name,played,points,position,pulse_id,...,strength,strength_attack_away,strength_attack_home,strength_defence_away,strength_defence_home,strength_overall_away,strength_overall_home,team_division,unavailable,win
0,3,0,NaN,1,0,Arsenal,0,0,0,1,...,4,1170,1170,1200,1150,1240,1180,NaN,False,0
1,7,0,NaN,2,0,Aston Villa,0,0,0,2,...,2,980,970,1040,1000,1050,1020,NaN,False,0
2,91,0,NaN,3,0,Bournemouth,0,0,0,127,...,2,1030,990,1050,1000,1020,1020,NaN,False,0
3,36,0,NaN,4,0,Brighton,0,0,0,131,...,2,1100,1100,1030,1040,1010,1050,NaN,False,0
4,90,0,NaN,5,0,Burnley,0,0,0,43,...,3,1070,1130,1110,970,1180,1110,NaN,False,0
5,8,0,NaN,6,0,Chelsea,0,0,0,4,...,4,1240,1240,1250,1220,1230,1220,NaN,False,0
6,31,0,NaN,7,0,Crystal Palace,0,0,0,6,...,3,1180,1080,1040,1040,1120,1130,NaN,False,0
7,11,0,NaN,8,0,Everton,0,0,0,7,...,3,1210,1100,1180,1170,1240,1100,NaN,False,0
8,13,0,NaN,9,0,Leicester,0,0,0,26,...,3,1080,1150,1100,1210,1160,1210,NaN,False,0
9,14,0,NaN,10,0,Liverpool,0,0,0,10,...,5,1320,1310,1330,1310,1350,1340,NaN,False,0


In [64]:
teams_df.columns

Index(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points',
       'position', 'pulse_id', 'short_name', 'strength',
       'strength_attack_away', 'strength_attack_home', 'strength_defence_away',
       'strength_defence_home', 'strength_overall_away',
       'strength_overall_home', 'team_division', 'unavailable', 'win'],
      dtype='object')

In [65]:
gw_data_df_1920["name"] = gw_data_df_1920["team"]

In [66]:
gw_data_1920_df = pd.merge(gw_data_df_1920, teams_df, how="inner", on="name")\
[["id", "team", "fixture", "gw", "fdr", "code", "pulse_id"]]

In [67]:
gw_data_1920_df.to_csv("../data/vaastav/gw_data_1920.csv")

In [71]:
gw_data_1920_df

,id,team,fixture,gw,fdr,code,pulse_id
0,1,Arsenal,a,gw_1,2,3,1
1,1,Arsenal,h,gw_10,3,3,1
2,1,Arsenal,h,gw_11,4,3,1
3,1,Arsenal,a,gw_12,3,3,1
4,1,Arsenal,h,gw_13,3,3,1
...,...,...,...,...,...,...,...
527,20,Wolves,h,gw_5,4,39,38
528,20,Wolves,a,gw_6,3,39,38
529,20,Wolves,h,gw_7,2,39,38
530,20,Wolves,a,gw_8,4,39,38


In [76]:
gw_data_1920_df["gameweek"] = gw_data_1920_df.gw.apply(lambda x: int(x.split("_")[1]))

In [91]:
gw_data_1920_df.rename(columns={"team": "team_name", "id": "team"}, inplace=True)

In [96]:
gw_data_1920_df["common_id"] = gw_data_1920_df.apply(lambda x: f"{str(x['team'])}_{str(x['gameweek'])}", axis=1)

In [109]:
gw_data_1920_df.team.sort_values().unique()

array([ 1,  3,  4,  5,  6,  8,  9, 10, 13, 14, 16, 17, 18, 20])

In [81]:
players_1920_id_df.rename(columns={"GW": "gameweek"}, inplace=True)

In [82]:
players_1920_id_df.gameweek.replace({39: 30, 40: 31, 41: 32, 42: 33, 43: 34, 44: 35, 45: 36, 46: 37, 47: 38}, inplace=True)

In [98]:
players_1920_with_team_det["common_id"] = players_1920_with_team_det.apply(lambda x: f"{str(x['team'])}_{str(x['gameweek'])}", axis=1)

In [99]:
players_1920_with_team_det

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,value,was_home,yellow_cards,gameweek,team,team_code,id,first_name,second_name,common_id
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,...,50,True,0,1,19,21,376,Aaron,Cresswell,19_1
1,Aaron_Cresswell_376,0,0,0,0,0.0,376,13,0,0,...,50,False,0,2,19,21,376,Aaron,Cresswell,19_2
2,Aaron_Cresswell_376,0,0,0,0,0.0,376,29,0,0,...,50,False,0,3,19,21,376,Aaron,Cresswell,19_3
3,Aaron_Cresswell_376,0,0,0,0,0.0,376,40,0,0,...,49,True,0,4,19,21,376,Aaron,Cresswell,19_4
4,Aaron_Cresswell_376,0,0,0,0,0.0,376,41,0,0,...,49,False,0,5,19,21,376,Aaron,Cresswell,19_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,Joel_Mumbongo_665,0,0,0,0,0.0,665,366,0,0,...,45,False,0,37,5,90,665,Joel,Mumbongo,5_37
22556,Joel_Mumbongo_665,0,0,0,0,0.0,665,372,0,0,...,45,True,0,38,5,90,665,Joel,Mumbongo,5_38
22557,John-Kymani_Gordon_668,0,0,0,0,0.0,668,370,0,0,...,45,False,0,37,7,31,668,John-Kymani,Gordon,7_37
22558,John-Kymani_Gordon_668,0,0,0,0,0.0,668,374,0,0,...,45,True,0,38,7,31,668,John-Kymani,Gordon,7_38


In [102]:
players_1920_with_team_det.team.sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [103]:
players_1920_with_team_det.query("common_id == '19_1'")

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,value,was_home,yellow_cards,gameweek,team,team_code,id,first_name,second_name,common_id
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,...,50,True,0,1,19,21,376,Aaron,Cresswell,19_1
609,Albian_Ajeti_514,0,0,0,0,0.0,514,8,0,0,...,60,True,0,1,19,21,514,Albian,Ajeti,19_1
685,Alexandre Nascimento_Costa Silva_387,0,0,0,0,0.0,387,8,0,0,...,45,True,0,1,19,21,387,Alexandre Nascimento,Costa Silva,19_1
1256,Andriy_Yarmolenko_392,0,0,0,0,0.0,392,8,0,0,...,60,True,0,1,19,21,392,Andriy,Yarmolenko,19_1
1446,Angelo_Ogbonna_377,0,0,0,0,0.0,377,8,0,0,...,45,True,0,1,19,21,377,Angelo,Ogbonna,19_1
1750,Arthur_Masuaku_378,0,0,0,0,0.0,378,8,0,0,...,45,True,0,1,19,21,378,Arthur,Masuaku,19_1
2398,Ben_Johnson_384,0,0,0,0,0.0,384,8,0,0,...,40,True,0,1,19,21,384,Ben,Johnson,19_1
3009,Carlos_Sánchez_397,0,0,0,0,0.0,397,8,0,0,...,45,True,0,1,19,21,397,Carlos,Sánchez,19_1
4992,Declan_Rice_395,0,0,15,0,1.8,395,8,5,0,...,50,True,0,1,19,21,395,Declan,Rice,19_1
6211,Fabián_Balbuena_382,0,0,6,0,0.4,382,8,5,0,...,45,True,1,1,19,21,382,Fabián,Balbuena,19_1


In [104]:
gw_data_1920_df.query("common_id == '19_1'")

,team,team_name,fixture,gw,fdr,code,pulse_id,gameweek,common_id


In [100]:
pd.merge(players_1920_with_team_det, gw_data_1920_df, how="left", on="common_id")

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture_x,goals_conceded,goals_scored,...,second_name,common_id,team_y,team_name,fixture_y,gw,fdr,code,pulse_id,gameweek_y
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,...,Cresswell,19_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron_Cresswell_376,0,0,0,0,0.0,376,13,0,0,...,Cresswell,19_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aaron_Cresswell_376,0,0,0,0,0.0,376,29,0,0,...,Cresswell,19_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aaron_Cresswell_376,0,0,0,0,0.0,376,40,0,0,...,Cresswell,19_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aaron_Cresswell_376,0,0,0,0,0.0,376,41,0,0,...,Cresswell,19_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22685,Joel_Mumbongo_665,0,0,0,0,0.0,665,366,0,0,...,Mumbongo,5_37,5.0,Burnley,a,gw_37,2.0,90.0,43.0,37.0
22686,Joel_Mumbongo_665,0,0,0,0,0.0,665,372,0,0,...,Mumbongo,5_38,5.0,Burnley,h,gw_38,2.0,90.0,43.0,38.0
22687,John-Kymani_Gordon_668,0,0,0,0,0.0,668,370,0,0,...,Gordon,7_37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22688,John-Kymani_Gordon_668,0,0,0,0,0.0,668,374,0,0,...,Gordon,7_38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
